[gensim tutorial 2](https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html)

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [3]:
from pprint import pprint
from collections import defaultdict

stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [4]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('./deerwester.dict')
print(dictionary)

2020-11-06 07:00:12,847 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-11-06 07:00:12,849 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2020-11-06 07:00:12,850 : INFO : saving Dictionary object under ./deerwester.dict, separately None
2020-11-06 07:00:12,859 : INFO : saved ./deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [5]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [6]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('./deerwester.mm', corpus)
print(corpus)

2020-11-06 07:00:12,907 : INFO : storing corpus in Matrix Market format to ./deerwester.mm
2020-11-06 07:00:12,918 : INFO : saving sparse matrix to ./deerwester.mm
2020-11-06 07:00:12,920 : INFO : PROGRESS: saving document #0
2020-11-06 07:00:12,921 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2020-11-06 07:00:12,926 : INFO : saving MmCorpus index to ./deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [8]:
pprint(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


In [9]:
from smart_open import open

class MyCorpus:
    def __iter__(self):
        for line in open('https://radimrehurek.com/gensim_3.8.3/auto_examples/core/mycorpus.txt'):
            yield dictionary.doc2bow(line.lower().split())

In [10]:
corpus_memory_friendly = MyCorpus()
print(corpus_memory_friendly)

In [11]:
for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [12]:
dictionary = corpora.Dictionary(line.lower().split() for line in open('https://radimrehurek.com/gensim_3.8.3/auto_examples/core/mycorpus.txt'))
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()
print(dictionary)

2020-11-06 07:00:13,427 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-11-06 07:00:13,430 : INFO : built Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...) from 9 documents (total 69 corpus positions)


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [13]:
corpus = [[(1, 0.5)], []]
corpora.MmCorpus.serialize('./corpus.mm', corpus)

2020-11-06 07:00:13,454 : INFO : storing corpus in Matrix Market format to ./corpus.mm
2020-11-06 07:00:13,465 : INFO : saving sparse matrix to ./corpus.mm
2020-11-06 07:00:13,467 : INFO : PROGRESS: saving document #0
2020-11-06 07:00:13,470 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2020-11-06 07:00:13,475 : INFO : saving MmCorpus index to ./corpus.mm.index


In [14]:
corpora.SvmLightCorpus.serialize('./corpus.svmlight', corpus)
corpora.BleiCorpus.serialize('./corpus.lda-c', corpus)
corpora.LowCorpus.serialize('./corpus.low', corpus)

2020-11-06 07:00:13,496 : INFO : converting corpus to SVMlight format: ./corpus.svmlight
2020-11-06 07:00:13,508 : INFO : saving SvmLightCorpus index to ./corpus.svmlight.index
2020-11-06 07:00:13,520 : INFO : no word id mapping provided; initializing from corpus
2020-11-06 07:00:13,523 : INFO : storing corpus in Blei's LDA-C format into ./corpus.lda-c
2020-11-06 07:00:13,532 : INFO : saving vocabulary of 2 words to ./corpus.lda-c.vocab
2020-11-06 07:00:13,540 : INFO : saving BleiCorpus index to ./corpus.lda-c.index
2020-11-06 07:00:13,550 : INFO : no word id mapping provided; initializing from corpus
2020-11-06 07:00:13,552 : INFO : storing corpus in List-Of-Words format into ./corpus.low
2020-11-06 07:00:13,560 : WARNING : List-of-words format can only save vectors with integer elements; 1 float entries were truncated to integer value
2020-11-06 07:00:13,563 : INFO : saving LowCorpus index to ./corpus.low.index


In [15]:
corpus = corpora.MmCorpus('./corpus.mm')

2020-11-06 07:00:13,593 : INFO : loaded corpus index from ./corpus.mm.index
2020-11-06 07:00:13,596 : INFO : initializing cython corpus reader from ./corpus.mm
2020-11-06 07:00:13,603 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


In [16]:
print(corpus)

MmCorpus(2 documents, 2 features, 1 non-zero entries)


In [17]:
print(list(corpus))

[[(1, 0.5)], []]


In [18]:
for doc in corpus:
    print(doc)

[(1, 0.5)]
[]


In [19]:
corpora.BleiCorpus.serialize('./corpus.lda-c', corpus)

2020-11-06 07:00:13,662 : INFO : no word id mapping provided; initializing from corpus
2020-11-06 07:00:13,666 : INFO : storing corpus in Blei's LDA-C format into ./corpus.lda-c
2020-11-06 07:00:13,678 : INFO : saving vocabulary of 2 words to ./corpus.lda-c.vocab
2020-11-06 07:00:13,687 : INFO : saving BleiCorpus index to ./corpus.lda-c.index


In [20]:
import gensim
import numpy as np

numpy_matrix = np.random.randint(10, size=[5, 2])
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)

In [21]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5, 2)
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)